## Old Versions

In [2]:
import pandas as pd
import os
from datetime import date
import xml.etree.ElementTree as ET

database_path = "nasil_olunur_database.xlsx"
saf_root = "SAF/"

### Creating contents file

In [5]:
# Create the contents:
def create_contents(ep_no):
    # path = saf_root + str(ep_no)
    path = os.path.join(saf_root, str(ep_no))

    text = ""
    for item in os.listdir(path):
        text += item +"\n"

    # save it into a file in correct dir.
    with open(path+"/contents", "w") as f:
        f.write(text)

# create contents file for episodes 2&3
for no in episode_nos:
    create_contents(no)


### Creating dublin_core.xml file

In [ ]:
# Create dublin_core
def create_dublin_core(ep_no):
    people = df[ep_no][people]

dublin_core_metadata_structure = {
    "contributor_author": guest,
    "date_accessioned": str(date.today),
    "date_issued": episode_date,
    "identifier_uri": rss_uri,
    "description_abstract_en": description_en,
    "description_abstract_tr": description_tr,
    "language_iso": "tr",
    "subjects": {
        "en": subjects_en,
        "tr": subjects_tr,
    },
    "title": episode_title,
    "type": "Recording, oral",
}


In [36]:
# guest = "Zeynep Atakan"
# episode_title = "2. Bölüm - Zeynep Atakan"
# date = date.today()
# episode_date = "2019-03-28"
# rss_uri = "https://feeds.simplecast.com/TRRbx_5C"

guest = "İsmail Küçükkaya"
episode_title = "3. Bölüm - İsmail Küçükkaya"
episode_date = "2019-03-28"
rss_uri = "https://feeds.simplecast.com/TRRbx_5C"

In [3]:
dublin_core_metadata_structure = {
    "contributor_author": guest,
    "date_accessioned": "2024-12-20",
    "date_issued": episode_date,
    "identifier_uri": rss_uri,
    "description_abstract_en": description_en,
    "description_abstract_tr": description_tr,
    "language_iso": "tr",
    "subjects": {
        "en": subjects_en,
        "tr": subjects_tr,
    },
    "title": episode_title,
    "type": "Recording, oral",
}

NameError: name 'guest' is not defined

In [39]:
# Create the root element
root = ET.Element("dublin_core", attrib={"schema": "dc"})

# Add metadata as sub-elements
ET.SubElement(root, "dcvalue", attrib={"element": "contributor", "qualifier": "author"}).text = dublin_core_metadata_structure["contributor_author"]
ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "accessioned"}).text = dublin_core_metadata_structure["date_accessioned"]
ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "issued"}).text = dublin_core_metadata_structure["date_issued"]
ET.SubElement(root, "dcvalue", attrib={"element": "identifier", "qualifier": "uri"}).text = dublin_core_metadata_structure["identifier_uri"]

# Add descriptions
ET.SubElement(
    root,
    "dcvalue",
    attrib={"element": "description", "qualifier": "abstract", "language": "en"}
).text = dublin_core_metadata_structure["description_abstract_en"]
ET.SubElement(
    root,
    "dcvalue",
    attrib={"element": "description", "qualifier": "abstract", "language": "tr"}
).text = dublin_core_metadata_structure["description_abstract_tr"]

# Add subjects
for subject in dublin_core_metadata_structure["subjects"]["en"]:
    ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none", "language": "en"}).text = subject

for subject in dublin_core_metadata_structure["subjects"]["tr"]:
    ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none", "language": "tr"}).text = subject

# Add title and type
ET.SubElement(root, "dcvalue", attrib={"element": "title", "qualifier": "none"}).text = dublin_core_metadata_structure["title"]
ET.SubElement(root, "dcvalue", attrib={"element": "type", "qualifier": "none"}).text = dublin_core_metadata_structure["type"]

# Write to an XML file
tree = ET.ElementTree(root)
tree.write("dublin_core.xml", encoding="utf-8", xml_declaration=True)

# Write to an XML file in the SAF/2 directory
output_dir = "../SAF/3"
output_file = os.path.join(output_dir, "dublin_core.xml")
tree = ET.ElementTree(root)
tree.write(output_file, encoding="utf-8", xml_declaration=True)

print("XML file created: dublin_core.xml")

XML file created: dublin_core.xml


#### Yapilacaklar
* episode date
* rss uri
* tr description
* en description kaydedilecek
* subject'lerin elle yazilmasi icin yontem gelistirilecek

** database formati guncellenecek, dogru formatlar yuklenecek
** LLM agentlari ile sistem kurulup anahtar kelime buldurtulacak
** SAF dosyalari yaratma islemi otomatik hale getirilecek

### Creating metadata_local.xml

In [4]:
# Dillerin otomatik belirlenmesini ayarlamam lazim...
def create_xml_file():
    # Create the root element
    root = ET.Element("dublin_core", schema="local")

    # Add people to the XML
    for person in people:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "person")
        dcvalue.set("qualifier", "name")
        dcvalue.set("language", "tr")
        dcvalue.text = person

    # Add places to the XML
    for place in places:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "place")
        dcvalue.set("qualifier", "name")
        dcvalue.set("language", "tr")
        dcvalue.text = place

    # Create an ElementTree object
    tree = ET.ElementTree(root)

    output_dir = "../SAF/2"
    output_file = os.path.join(output_dir, "metadata_local.xml")
    # Write the tree to an XML file
    with open(output_file, "wb") as file:
        file.write(b'<?xml version="1.0" encoding="utf-8" standalone="no"?>\n')
        tree.write(file, encoding="utf-8", xml_declaration=False)

# Generate the XML file
create_xml_file()
print("XML file created as 'output.xml'.")

NameError: name 'people' is not defined

# SAF Creator Polished

### Workflow:

* Create Episode Folder
* Add Episode Document
* Create & Populate Contents File
* Create & Populate dublin_core.xml File
* Create & Populate metadata_local.xml File
* Download & add the episode audio file

Database Format:

Columns:
- Episode Number (key)
- Episode Title
- Episode Audio URL
- Guest Name
- Word Document Name
- Keyword - People
- Keyword - Places (TR)
- Keyword - Subjects (TR)
- Keyword - Subjects (EN)

## SAF Functions

In [36]:
import pandas as pd
import os
from datetime import date
import xml.etree.ElementTree as ET
import shutil
import subprocess


saf_root = "SAF/"
database_path = "nasil_olunur_database.xlsx"

episode_numbers = [str(i) for i in range(59,61)] # remove this later on

# episode_numbers = [
#     4, 5, 6, 7, 8, 9, 11, 12, 14, 15, 17, 19, 21, 22, 23, 24, 25, 26, 27,
#     28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
#     46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58
# ]

In [37]:
"""
Executing creating SAF folders per episode
"""

for ep in episode_numbers:
    create_SAF_folder(ep)


IndexError: single positional indexer is out-of-bounds

In [35]:
"""
Function to create the SAF file for the episode

Calls functions to create contents, dublin_core, metadata files and download episode audio.
"""
def create_SAF_folder(episode_number):
    df = pd.read_excel(database_path)
    # row = df.loc[df["Episode Number"] == episode_number]
    row = df.loc[df["Episode Number"] == episode_number].iloc[0]
    title = str(episode_number) + "-" + row["Guest Name"]
    url = row["Episode Audio Url"]

    episode_folder_path = saf_root + str(episode_number)

    # Create the folder
    os.makedirs(episode_folder_path, exist_ok=True)

    # Add the docx file
    start_path = row["Word Document Name"]
    add_docx(start_path, episode_folder_path)

    # Add the audio file
    # add_audio(title,url,episode_folder_path)

    # Create the contents file
    create_contents(episode_folder_path)

    # Create the dublin_core
    create_dublin_core(episode_number, episode_folder_path)

    # Create the metadata
    create_metadata(episode_number,episode_folder_path)

In [27]:
"""
Function to create contents file for the episode
"""
def create_contents(path):
    text = ""
    for item in os.listdir(path):
        text += item + "\t bundle:ORIGINAL" +"\n"

    # save it into a file in correct dir.
    with open(path+"/contents", "w") as f:
        f.write(text)

In [28]:
"""
    Function to copy Word Document to SAF Folder
"""
def add_docx(start_path, end_path):
    shutil.copy2(start_path, end_path)

In [29]:
"""
    Function to download audio file
"""
def add_audio(title, url,episode_folder_path):

    filename = os.path.join(episode_folder_path, f"{title}.mp3")

    ffmpeg_command = [
        "ffmpeg", "-i", url,                # Input URL
        "-b:a", "64k",                      # Set audio bitrate
        "-f", "mp3",                        # Output format
        "-y",                               # Overwrite existing file
        "-loglevel", "error",               # Suppress all but error output
        filename                            # Output file path
    ]
    try:
        subprocess.run(ffmpeg_command, check=True)
        print(f"Downloaded and compressed: {title}")
    except subprocess.CalledProcessError:
        print(f"Failed to download and compress: {title}")

In [30]:
"""
    Function to create dublin_core file
"""
rss_uri = "https://feeds.simplecast.com/TRRbx_5C"

def create_dublin_core(episode_number, episode_folder_path):
    df = pd.read_excel(database_path)
    episode_row = df.loc[df["Episode Number"] == episode_number, ["Guest Name", "Episode Date", "Description (TR)", "Keyword - Subjects (TR)", "Keyword - Subjects (EN)", "Episode Title"]]

    if episode_row.empty:
        raise ValueError(f"No episode found with Episode Number: {episode_number}")

    # Extract scalar values
    guest = episode_row["Guest Name"].iloc[0]
    episode_date = episode_row["Episode Date"].iloc[0]
    description_tr = episode_row["Description (TR)"].iloc[0]
    episode_title = episode_row["Episode Title"].iloc[0]

    # Extract list values
    subjects_en = episode_row["Keyword - Subjects (EN)"].iloc[0].split(",") if pd.notnull(episode_row["Keyword - Subjects (EN)"].iloc[0]) else []
    subjects_tr = episode_row["Keyword - Subjects (TR)"].iloc[0].split(",") if pd.notnull(episode_row["Keyword - Subjects (TR)"].iloc[0]) else []

    dublin_core_metadata_structure = {
        "contributor_author": guest,
        "date_accessioned": str(date.today()),
        "date_issued": str(episode_date),
        "identifier_uri": rss_uri,
        "description_abstract_en": "None",
        "description_abstract_tr": description_tr,
        "language_iso": "tr",
        "subjects": {
            "en": subjects_en,
            "tr": subjects_tr,
        },
        "title": episode_title,
        "type": "Recording, oral",
    }

    # Create the root element
    root = ET.Element("dublin_core", attrib={"schema": "dc"})

    # Add metadata as sub-elements
    ET.SubElement(root, "dcvalue", attrib={"element": "contributor", "qualifier": "author"}).text = dublin_core_metadata_structure["contributor_author"]
    ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "accessioned"}).text = dublin_core_metadata_structure["date_accessioned"]
    ET.SubElement(root, "dcvalue", attrib={"element": "date", "qualifier": "issued"}).text = dublin_core_metadata_structure["date_issued"]
    ET.SubElement(root, "dcvalue", attrib={"element": "identifier", "qualifier": "uri"}).text = dublin_core_metadata_structure["identifier_uri"]

    # Add descriptions
    ET.SubElement(
        root,
        "dcvalue",
        attrib={"element": "description", "qualifier": "abstract", "language": "en"}
    ).text = dublin_core_metadata_structure["description_abstract_en"]
    ET.SubElement(
        root,
        "dcvalue",
        attrib={"element": "description", "qualifier": "abstract", "language": "tr"}
    ).text = dublin_core_metadata_structure["description_abstract_tr"]

    # Add subjects
    for subject in dublin_core_metadata_structure["subjects"]["en"]:
        ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none", "language": "en"}).text = subject

    for subject in dublin_core_metadata_structure["subjects"]["tr"]:
        ET.SubElement(root, "dcvalue", attrib={"element": "subject", "qualifier": "none", "language": "tr"}).text = subject

    # Add title and type
    ET.SubElement(root, "dcvalue", attrib={"element": "title", "qualifier": "none"}).text = dublin_core_metadata_structure["title"]
    ET.SubElement(root, "dcvalue", attrib={"element": "type", "qualifier": "none"}).text = dublin_core_metadata_structure["type"]

    # Write to an XML file
    output_file = os.path.join(episode_folder_path, "dublin_core.xml")
    tree = ET.ElementTree(root)
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

    print(f"XML file created: {output_file}")

In [31]:
"""
    Function to create metadata_local file
"""
def create_metadata(episode_number,episode_folder_path):

    episode_row = df.loc[df["Episode Number"] == episode_number, ["Keyword - People","Keyword - Places (TR)"]]

    if episode_row.empty:
        raise ValueError(f"No episode found with Episode Number: {episode_number}")

    # Extract list values
    people = episode_row["Keyword - People"].iloc[0].split(",") if pd.notnull(episode_row["Keyword - People"].iloc[0]) else []
    places = episode_row["Keyword - Places (TR)"].iloc[0].split(",") if pd.notnull(episode_row["Keyword - Places (TR)"].iloc[0]) else []


    # Create the root element
    root = ET.Element("dublin_core", schema="local")

    # Add people to the XML
    for person in people:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "person")
        dcvalue.set("qualifier", "name")
        dcvalue.set("language", "tr")
        dcvalue.text = person

    # Add places to the XML
    for place in places:
        dcvalue = ET.SubElement(root, "dcvalue")
        dcvalue.set("element", "place")
        dcvalue.set("qualifier", "name")
        dcvalue.set("language", "tr")
        dcvalue.text = place

    # Create an ElementTree object
    tree = ET.ElementTree(root)

    output_dir = episode_folder_path
    output_file = os.path.join(output_dir, "metadata_local.xml")
    # Write the tree to an XML file
    with open(output_file, "wb") as file:
        file.write(b'<?xml version="1.0" encoding="utf-8" standalone="no"?>\n')
        tree.write(file, encoding="utf-8", xml_declaration=False)